# <span style="color:teal">RBFE Network Setup</span>



In [ ]:
# import libraries
import BioSimSpace as BSS
import os
import sys
import glob
import csv
import numpy as np
import networkx as nx
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
import itertools

from pipeline.prep import *
from pipeline.utils import *

pipeline.__file__

In [ ]:
# now want to start putting it all together by initialising the pipeline
# this is so can have all the file locations 

pl = initialise_pipeline()
# where the ligands for the pipeline are located. These should all be in the same folder in sdf format
pl.ligands_folder(f"/home/anna/Documents/benchmark/inputs/tyk2/ligands")
# where the pipeline should be made
pl.main_folder("/home/anna/Documents/code/write/test")


#### <span style="color:teal">2. Setting up the Network</span>
<a id="setup"></a>

##### <span style="color:teal">Choosing the parameters for the FEP runs</span>
<a id="parameters"></a>


In [ ]:
#nodes to pick things
node = BSS.Gateway.Node("A node to create input files for molecular dynamics simulation.")

node.addInput("Ligand FF", BSS.Gateway.String(help="Force field to parameterise ligands with.",
                                             allowed=["GAFF2", "Parsely", "Sage"],
                                             default="Sage"))

node.addInput("Protein FF", BSS.Gateway.String(help="Force field to parameterise the protein with.",
                                             allowed=["ff03", "ff14SB", "ff99", "ff99SB", "ff99SBildn"],
                                             default="ff14SB"))

node.addInput("Water Model", BSS.Gateway.String(help="Water model to use.",
                                             allowed=["SPC", "SPCE", "TIP3P", "TIP4P", "TIP5P"],
                                             default="TIP3P"))

node.addInput("Box Edges", BSS.Gateway.String(help="Size of water box around molecular system.",
                                             allowed=["20*angstrom", "25*angstrom", "30*angstrom", "35*angstrom", "45*angstrom", "5*nm", "7*nm", "10*nm"],
                                             default="30*angstrom"))

node.addInput("Box Shape", BSS.Gateway.String(help="Geometric shape of water box.",
                                             allowed=["cubic", "truncatedOctahedron"],
                                             default="truncatedOctahedron"))

node.addInput("Run Time", BSS.Gateway.String(help="The sampling time per lambda window.",
                                             allowed=["10*ps", "100*ps", "1*ns", "2*ns", "3*ns", "4*ns", "5*ns", "8*ns", "10*ns", "12*ns", "15*ns"],
                                             default="4*ns"))

node.addInput("HMR", BSS.Gateway.String(help="Whether or not Hydrogen Mass repartitioning should be used. If true, a timestep of 4 fs will be used.",
                                             allowed=["True","False"],
                                             default="True"))

engines_options = [e.upper() for e in BSS.FreeEnergy.engines()]
engines_options.append("ALL")

node.addInput("FEP Engine", BSS.Gateway.String(help="Engine to run FEP with. BSS available engines, or ALL.",
                                             allowed=engines_options,
                                             default="ALL"))

node.addInput("LambdaWindows", BSS.Gateway.String(help="The number of lambda windows for regular transformations.",
                                             allowed=["3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"],
                                             default="11"))

# node.addInput("DiffLambdaWindows", BSS.Gateway.String(help="The number of lambda windows for difficult transformations.",
#                                              allowed=["4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"],
#                                              default="17"))
                                             
# node.addInput("LOMAP Threshold", BSS.Gateway.String(help="The LOMAP score threshold to define difficult transformations.",
#                                              allowed=["0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9"],
#                                              default="0.4"))

node.addInput("Number of repeats", BSS.Gateway.String(help="The number of repeats of the simulation.",
                                             allowed=[str(i) for i in range (1,11)],
                                             default=str(3)))

node.addInput("Keep trajectories", BSS.Gateway.String(help="Whether to keep the trajectory files or not.",
                                             allowed=["None","0,0.5,1", "0,1", "All"],
                                             default="0,0.5,1"))

node.showControls()


In [ ]:
# create protocol dictionary
protocol_dict = {
    "ligand forcefield":node.getInput('Ligand FF'),
    "protein forcefield":node.getInput('Protein FF'),
    "solvent":node.getInput('Water Model'),
    "box edges":node.getInput('Box Edges').split("*")[0],
    "box edges unit":node.getInput('Box Edges').split("*")[1],
    "box type":node.getInput('Box Shape'),
    "sampling":node.getInput('Run Time').split("*")[0],
    "sampling unit":node.getInput('Run Time').split("*")[1],
    "hmr":node.getInput('HMR'),
    "repeats":node.getInput('Number of repeats'),
    "trajectories":node.getInput('Keep trajectories'),
}

# use pipeline protocol to check this
protocol = pipeline_protocol(protocol_dict, verbose=False)
# validate the given inputs and provide defaults
protocol.validate()
# print to check that all the settings are what is wanted
print("current protocol:")
protocol.print_protocol()

In [ ]:
# change any protocol settings
# this can be done eg
protocol.ligand_forcefield("GAff2")
protocol.start_temperature(34)

# can also call all protocol options as a dictionary
protocol.dictionary()

In [ ]:
#nodes to pick things
ana_node = BSS.Gateway.Node("A node to specify the analysis for molecular dynamics simulation.")

ana_node.addInput("estimator", BSS.Gateway.String(help="Estimator used to analyse.",
                                             allowed=["MBAR", "TI"],
                                             default="MBAR"))

ana_node.addInput("method", BSS.Gateway.String(help="method used to analyse.",
                                             allowed=["alchemlyb", "native"],
                                             default="alchemlyb"))

ana_node.showControls()


In [ ]:
# create protocol dictionary
ana_protocol_dict = {
    "estimator":ana_node.getInput('estimator'),
    "method":ana_node.getInput('method'),
}

# use pipeline protocol to check this
ana_protocol = analysis_protocol(ana_protocol_dict, verbose=True)
# validate the given inputs and provide defaults
ana_protocol.validate()
# print to check that all the settings are what is wanted
print("current analysis protocol:")
ana_protocol.print_protocol()

In [ ]:
# can also change things in the analysis protocol as well, eg
ana_protocol.truncate_keep("end")
ana_protocol.truncate_percentage("5")

# check again using
ana_protocol.print_protocol()
# or
ana_protocol.dictionary()

##### <span style="color:teal">Protein Parameterisation</span>  


##### <span style="color:teal">Protein parameterisation</span>  

This needs to be carried out carefully.

can parameterise using :
```python
prot = BSS.IO.readPDB(path_to_protein, pdb4amber=False)[0]
prot_p = BSS.Parameters.parameterise(prot, protocol.protein_forcefield()).getMolecule()
BSS.IO.saveMolecules("inputs/protein", prot_p, ["PRM7","RST7"])
```

tleap may fail. Best to parameterise carefully before and also consider crystal waters.

can view using:
```python
BSS.Notebook.View(f"{input_dir}/{protein}/protein/{protein}_parameterised.pdb").system()
```


In [ ]:
# add the protein file locations to the pipeline setup object
pl.protein_path(f"/home/anna/Documents/benchmark/inputs/tyk2/tyk2_parameterised")


##### <span style="color:teal">The FEP Network</span>  


In [ ]:
# now want to read in all the ligands in the earlier provided ligands folder
pl.setup_ligands()
# this should write a ligands.dat in the execution model.

In [ ]:
# to remove a ligand, can do the following:
pl.remove_ligand("lig_ejm44")

# if you want to add a ligand instead, eg an intermediate, current method is to copy it to the folder and rerun the ligands setup

In [ ]:
# next, want to setup a network
pl.setup_network()

In [ ]:
# it is possible to remove a perturbation using:
# (the order of the ligands must be correct, can check using pl.perturbations)
pl.remove_perturbation("lig_ejm31~lig_ejm48")

# can also add a perturbation
pl.add_perturbation("lig_jmc30~lig_ejm45")

# redraw the network using 
pl.draw_network()

##### <span style="color:teal">Preparing for the FEP pipeline</span>  


In [ ]:
# add the protocols to the pipeline setup. This will also write them.
pl.add_pipeline_protocol(protocol)
pl.add_analysis_protocol(ana_protocol)

In [ ]:
# write the run_all script, also does a final ligand and network write
pl.write_run_all()

##### <span style="color:teal">Generating the RBFENN</span>  


how to use diff links file for the network generation.

In [ ]:
# for sem perturbations
tgt_to_run = f"{protein}_rename" #f"{protein}_me" f"{protein}_rename" for tyk2 and p38
cats_files_path = f"{main_folder}/scripts/RBFENN/ANALYSIS/perturbation_networks/output/series_predictions"

In [ ]:
def scaleArray(arr):
    """Scales an array to be the inverse in the range [0-1]."""
    
    # normalise to the range 0-1.
    return minmax_scale(1 /  arr, feature_range=(0,1))


In [ ]:
# get the FEPNN SEM prediction per ligand.
perts = {}
for cats_file in glob.glob(f"{cats_files_path}/{tgt_to_run}_*"):
    
    with open(cats_file, "r") as readfile:
        reader = csv.reader(readfile)
        next(reader)
        for row in reader:
            pert = row[0]
            pred_sem = float(row[1])
            
            if not pert in perts:
                perts[pert] = [pred_sem]
            else:
                perts[pert].append(pred_sem)
            
# compute the mean SEM prediction per pert.
pert_names = []
pert_sems = []
for pert, sems in perts.items():
    mean_sem = np.mean(sems)
    pert_names.append(pert)
    pert_sems.append(float(mean_sem))

# now scale the sems to [0-1].
pert_sems = scaleArray(np.array(pert_sems))

for pert, val in zip(pert_names, pert_sems):
    perts[pert] = val

In [ ]:
# make folder for the RBFENN network
validate.folder_path(f"{pl.exec_folder()}/RBFENN", create=True)

written = []
with open(f"{pl.exec_folder()}/RBFENN/links_file.in", "w") as writefile:
    writer = csv.writer(writefile, delimiter =" ")
    
    for pert_name, value in perts.items():
        # find the lomap filename for both ligs.
        liga_lomap_name = None
        ligb_lomap_name = None
        for filename in glob.glob(f"{pl.ligands_folder()}/*.sdf"):
            # if "lig_8" in filename:
            #     continue # exclude +1 ligands from tnks2 set.
            if pert_name.split("~")[0] in filename:
                liga_lomap_name = filename.split("/")[-1].split(".")[0]#.replace("ejm","ejm_").replace("jmc","jmc_")
            elif pert_name.split("~")[1] in filename:
                ligb_lomap_name = filename.split("/")[-1].split(".")[0]#.replace("ejm","ejm_").replace("jmc","jmc_")
            
            if liga_lomap_name and ligb_lomap_name:
                if not [liga_lomap_name, ligb_lomap_name] in written:
                    writer.writerow([liga_lomap_name, ligb_lomap_name, value])
                    
                    written.append([liga_lomap_name, ligb_lomap_name])

In [ ]:
# ligands and ligands_names already exists due to lomap above
# if change the folder name, will put this in the execution model as default.
pl.setup_network(folder="RBFENN", links_file=f"{pl.exec_folder()}/RBFENN/links_file.in")

# this will update the existing network.
# write the rbfenn to a different network file
pl.write_network(file_path=f"{pl.exec_folder()}/rbfenn_network.dat")